Task 1: Third-order letter approximation model

In [163]:
files = [
    '/workspaces/emergingtechnologies/Data/Frankenstein.txt',
    '/workspaces/emergingtechnologies/Data/BookPlates.txt',
    '/workspaces/emergingtechnologies/Data/SpyglassMoutain.txt',
    '/workspaces/emergingtechnologies/Data/TheOldHouse.txt',
    '/workspaces/emergingtechnologies/Data/WindOfDestiny.txt',
]

texts = {}

#preamble and postamble markers
start_marker = " ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

#read each file and remove preamble and postamble
for file_path in files:
    with open(file_path, 'r') as f:
        text = f.read()

        #find the start and end markers
        start = text.find(start_marker)
        end = text.find(end_marker)


        #skip the start marker
        start += len(start_marker)
        #extract all text between the markers
        text = text[start:end].strip()

        #store the cleaned text in the dictionary with filename as the key
        filename = file_path.split('/')[-1]  # Get filename from path
        texts[filename] = text

In [164]:
#clean the text by removing all characters except for uppercase letters, spaces, and periods
chars = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ. ')
#convert the text to uppercase
text = text.upper()
#remove all characters that are not in the set of allowed characters
cleaned = ''.join(c for c in text if c in chars)


In [165]:
model = {}

#iterate over the text and count the occurrences of each trigram
for i in range(len(cleaned) - 1):
  digram = cleaned[i:i+3]
  model[digram] = model.get(digram, 0) + 1

In [166]:
sorted(model.items(), key=lambda x: x[1], reverse=True)

[('   ', 5001),
 (' TH', 2548),
 ('THE', 2034),
 ('HE ', 1749),
 (' I ', 954),
 ('ND ', 927),
 ('AND', 921),
 (' AN', 910),
 (' TO', 897),
 ('ED ', 829),
 ('ING', 798),
 ('TO ', 776),
 ('AT ', 718),
 ('ER ', 697),
 ('NG ', 692),
 (' OF', 650),
 ('OF ', 618),
 (' HA', 592),
 (' A ', 590),
 (' YO', 586),
 (' IN', 585),
 ('YOU', 582),
 ('IN ', 579),
 ('HAT', 566),
 ('E T', 559),
 (' HE', 534),
 ('HER', 530),
 ('THA', 509),
 ('AS ', 496),
 ('N T', 455),
 (' WA', 455),
 ('IS ', 451),
 ('EN ', 451),
 ('T T', 449),
 ('RE ', 446),
 ('ME ', 432),
 (' WH', 427),
 ('NT ', 419),
 ('. I', 417),
 ('E S', 406),
 ('D T', 405),
 (' IT', 393),
 ('E A', 393),
 (' LI', 388),
 ('VE ', 386),
 ('ES ', 384),
 ('E W', 379),
 (' BE', 376),
 ('S A', 372),
 ('ON ', 366),
 ('MY ', 366),
 ('IT ', 366),
 (' ME', 359),
 ('T I', 358),
 ('OU ', 356),
 ('UT ', 354),
 (' MY', 350),
 ('ERE', 349),
 (' WI', 343),
 ('LL ', 333),
 (' ON', 328),
 ('S T', 325),
 ('AVE', 321),
 (' CO', 313),
 ('GHT', 308),
 ('TER', 308),
 ('T A